In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType, StringType
#sc = SparkSession.builder.master("local[*]").getOrCreate()
#Spark Contexto
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

In [0]:
from pyspark.sql import SparkSession  
scSpark = SparkSession \
    .builder \
    .appName("Python Spark IBM") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

#sdfData.schema
#sdfData.printSchema()
#sdfData.count()
#sdfData.show()
#sdfData.groupby('idsensor').count().show()


In [4]:
df = scSpark.read.load('/content/a2.parquet')
df.createOrReplaceTempView('df')
scSpark.sql('SELECT * from df').show()

+---------+--------+-----------+-----------+--------+---------+-----------+------------+-------------------+-------------+--------------+--------------+-------------+-----------+-----+---------+
|   cidade|idsensor|tipo_sangue|data_coleta|latitude|longitude|hora_coleta|hora_entrega|       local_coleta|local_entrega|cpf_consumidor| transportador|      testado|temperatura|caixa|geladeira|
+---------+--------+-----------+-----------+--------+---------+-----------+------------+-------------------+-------------+--------------+--------------+-------------+-----------+-----+---------+
|Fortaleza|       1|          A| 2018-12-31|  123456|    98765|03:00:00 AM| 04:00:00 AM|                HGF|       HEMOCE|  9945JJKAHA95| KK8988SFGJKL4|CARRO 0383344|        SIM|   12|      100|
|  Caucaia|       2|          A| 2018-12-31|  123457|    98766|03:00:00 AM| 04:00:00 AM|HOSPITAL DO CORACAO|       HEMOCE|  9945JJKAHA96| KK8988SFGJKL5|CARRO 0383345|        SIM|   13|      101|
|Maracanau|       3|     

In [5]:
scSpark.sql('SELECT * from df').show(3)

+---------+--------+-----------+-----------+--------+---------+-----------+------------+-------------------+-------------+--------------+-------------+-------------+-----------+-----+---------+
|   cidade|idsensor|tipo_sangue|data_coleta|latitude|longitude|hora_coleta|hora_entrega|       local_coleta|local_entrega|cpf_consumidor|transportador|      testado|temperatura|caixa|geladeira|
+---------+--------+-----------+-----------+--------+---------+-----------+------------+-------------------+-------------+--------------+-------------+-------------+-----------+-----+---------+
|Fortaleza|       1|          A| 2018-12-31|  123456|    98765|03:00:00 AM| 04:00:00 AM|                HGF|       HEMOCE|  9945JJKAHA95|KK8988SFGJKL4|CARRO 0383344|        SIM|   12|      100|
|  Caucaia|       2|          A| 2018-12-31|  123457|    98766|03:00:00 AM| 04:00:00 AM|HOSPITAL DO CORACAO|       HEMOCE|  9945JJKAHA96|KK8988SFGJKL5|CARRO 0383345|        SIM|   13|      101|
|Maracanau|       3|          

In [16]:
%%time
scSpark.sql('SELECT * from df WHERE cidade = "Fortaleza"').toPandas()

CPU times: user 33.8 ms, sys: 11.7 ms, total: 45.5 ms
Wall time: 456 ms


,cidade,idsensor,tipo_sangue,data_coleta,latitude,longitude,hora_coleta,hora_entrega,local_coleta,local_entrega,cpf_consumidor,transportador,testado,temperatura,caixa,geladeira
0,Fortaleza,1,A,2018-12-31,123456,98765,03:00:00 AM,04:00:00 AM,HGF,HEMOCE,9945JJKAHA95,KK8988SFGJKL4,CARRO 0383344,SIM,12,100
1,Fortaleza,4,A,2018-12-31,123459,98768,03:00:00 AM,04:00:00 AM,HOSPITAL DO CORACAO,HEMOCE,9945JJKAHA98,KK8988SFGJKL7,CARRO 0383347,SIM,15,103
2,Fortaleza,7,A,2018-12-31,123462,98771,03:00:00 AM,04:00:00 AM,HGF,HEMOCE,9945JJKAHA101,KK8988SFGJKL10,CARRO 0383350,SIM,18,106
3,Fortaleza,10,A,2018-12-31,123465,98774,03:00:00 AM,04:00:00 AM,HOSPITAL DO CORACAO,HEMOCE,9945JJKAHA104,KK8988SFGJKL13,CARRO 0383353,SIM,21,109
4,Fortaleza,13,A,2018-12-31,123468,98777,03:00:00 AM,04:00:00 AM,HGF,HEMOCE,9945JJKAHA107,KK8988SFGJKL16,CARRO 0383356,SIM,24,112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1850,Fortaleza,5551,O,2018-12-31,129006,104315,09:00:00 AM,10:00:00 AM,HOSPITAL DA CRIANCA,HEMOCE,9945JJKAHA5645,KK8988SFGJKL5554,CARRO 0388894,NAO,31,5650
1851,Fortaleza,5554,O,2018-12-31,129009,104318,09:00:00 AM,10:00:00 AM,HOSPITAL DO CORACAO,HEMOCE,9945JJKAHA5648,KK8988SFGJKL5557,CARRO 0388897,NAO,34,5653
1852,Fortaleza,5557,O,2018-12-31,129012,104321,09:00:00 AM,10:00:00 AM,HOSPITAL DA CRIANCA,HEMOCE,9945JJKAHA5651,KK8988SFGJKL5560,CARRO 0388900,NAO,37,5656
1853,Fortaleza,5560,O,2018-12-31,129015,104324,09:00:00 AM,10:00:00 AM,HOSPITAL DO CORACAO,HEMOCE,9945JJKAHA5654,KK8988SFGJKL5563,CARRO 0388903,NAO,23,5659


In [0]:
df.write.parquet("cidade", mode='overwrite')

In [0]:
!rm -Rf a2_m1.json

In [0]:
df = df.repartition(1)
df.write.json('/content/a2_m1.json')

In [0]:
!rm -f rklib.py
!wget https://raw.githubusercontent.com/IBM/coursera/master/rklib.py

--2019-11-03 08:09:28--  https://raw.githubusercontent.com/IBM/coursera/master/rklib.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2540 (2.5K) [text/plain]
Saving to: ‘rklib.py’

rklib.py            100%[===================>]   2.48K  --.-KB/s    in 0s      

2019-11-03 08:09:28 (34.0 MB/s) - ‘rklib.py’ saved [2540/2540]



In [0]:

import zipfile

def zipdir(path, ziph):
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file))

zipf = zipfile.ZipFile('/content/a2_m1.json.zip', 'w', zipfile.ZIP_DEFLATED)
zipdir('/content/a2_m1.json', zipf)
zipf.close()

In [0]:
!base64 a2_m1.json.zip > a2_m1.json.zip.base64

In [0]:

from rklib import submit
key = "1injH2F0EeiLlRJ3eJKoXA"
part = "wNLDt"
email = "web2ajax@gmail.com"
secret = "6hz8tuE40BW10Zg2"

with open('/content/a2_m1.json.zip.base64', 'r') as myfile:
    data=myfile.read()
submit(email, secret, key, part, [part], data)

Submission successful, please check on the coursera grader page for the status
-------------------------
{"elements":[{"itemId":"O8C69","id":"f_F-qCtuEei_fRLwaVDk3g~O8C69~5gZUIP4aEem5bBKvLjOBZA","courseId":"f_F-qCtuEei_fRLwaVDk3g"}],"paging":{},"linked":{}}
-------------------------
